In [1]:
import re
import os
import copy
import random

import sys
print(sys.version)

from elasticsearch import Elasticsearch
from datetime import datetime, date, timedelta
import time
from creds import *
import pandas as pd

/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


3.5.2 (default, Nov 23 2017, 16:37:01) 
[GCC 5.4.0 20160609]


In [2]:
# Helper function for generating a client for connection to ES
def get_elastic_client(username, password):
    return Elasticsearch([{'host': 'elastic-analyst.ops.newknowledge.io', 'port': 5601}],http_auth=(username, password))

def get_data(es, query, index='discover_*'):
    results = es.search(index=index, body=query, scroll='5m', request_timeout=30)

    sc_id = results['_scroll_id']
    all_tweets = []
    num_res = 1
    start = datetime.now()
    try:
        i = 1
        while num_res > 0:
            print('Fetching batch', i)
            res = es.scroll(scroll_id=sc_id, scroll='2m', request_timeout=30)
            sc_id = res['_scroll_id']
            num_res = len(res['hits']['hits'])
            all_tweets.extend([r['_source'] for r in res['hits']['hits']])
            i += 1
    finally:
        es.clear_scroll(scroll_id=sc_id)

    return [(tweet['authorScreenName'], tweet['content'], tweet['createdAt'], tweet['hashtags'], tweet['domains'],
             tweet['urls'], tweet['parentScreenName'], tweet['authorUserId'], tweet['parentContentId'],
             tweet['parentUserId'], tweet['platform']) \
            for tweet in all_tweets if tweet['content'] and not tweet['connectionType'] == 'retweet']

def get_data_for_day_elastic(es=None, platform='twitter_stream', index='discover_*', project='*', start_day=0, end_day=0):
    if es == None:
        raise Exception("Elastic connection object not provided")

    query = {
        "query": {
            "bool":{
                "must":{
                    "term": {"project": project}
                },
                "filter": {
                    "range":{
                        "createdAt": {
                            "gte": 'now-{0}d/d'.format(start_day),
                            "lt": 'now-{0}d/d'.format(end_day)
                        }
                    }
                }
                
            }
        },
        "size": 10000
    }
    return get_data(es, query, index=index)



In [3]:
es = get_elastic_client('garrett','GYhc5xFvEpDcbG5x$n+yd')

In [4]:
start_time = time.time()
e_tweets = get_data_for_day_elastic(es, index='discover_business', project='hamiltonredux', start_day=7, end_day=0)
print(time.time() - start_time)

Traceback (most recent call last):
  File "/home/azunre/.local/lib/python3.5/site-packages/urllib3/connection.py", line 171, in _new_conn
    (self._dns_host, self.port), self.timeout, **extra_kw)
  File "/home/azunre/.local/lib/python3.5/site-packages/urllib3/util/connection.py", line 79, in create_connection
    raise err
  File "/home/azunre/.local/lib/python3.5/site-packages/urllib3/util/connection.py", line 69, in create_connection
    sock.connect(sa)
socket.timeout: timed out

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/azunre/.local/lib/python3.5/site-packages/elasticsearch/connection/http_urllib3.py", line 172, in perform_request
    response = self.pool.urlopen(method, url, body, retries=Retry(False), headers=request_headers, **kw)
  File "/home/azunre/.local/lib/python3.5/site-packages/urllib3/connectionpool.py", line 638, in urlopen
    _stacktrace=sys.exc_info()[2])
  File "/home/azunre/.local/lib/py

ConnectionError: ConnectionError((<urllib3.connection.HTTPConnection object at 0x7f23b920dfd0>, 'Connection to elastic-analyst.ops.newknowledge.io timed out. (connect timeout=30)')) caused by: ConnectTimeoutError((<urllib3.connection.HTTPConnection object at 0x7f23b920dfd0>, 'Connection to elastic-analyst.ops.newknowledge.io timed out. (connect timeout=30)'))

In [ ]:
e = pd.DataFrame(e_tweets)
e.columns = ['user_screen_name', 'text', 'created_at', 'hashtags', 'domains', 'urls', 'rt_screen_name', 'user_id_str', 'rt_id_str', 'rt_user_id_str', 'platform']

In [ ]:
e.head(10)

In [ ]:
len(e)

In [ ]:
e.to_csv('tweets_aug9_5days.csv', index=False)